In [ ]:
#mount the drive
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Verify the contents of the directory
# !ls /content/drive/MyDrive/NLP SEM 5


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/MyDrive/NLP SEM 5/CSVs/"


test_data  test_outputs  test_summaries  train_data  val_data


In [ ]:
!pip install -q transformers[torch] accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 59.9 MB/s eta 0:00:00


In [ ]:
!pip install -q tf-keras
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [ ]:
!pip install -q keras-nlp tensorflow-text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 29.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.15.1 requires tensorflow<2.16,>=2.15, but you have tensorflow 2.16.1 which is incompatibl

In [ ]:
# import os
# import pandas as pd
# from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, classification_report
# from tensorflow.keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
# from keras.models import Sequential
# from keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout
# from keras.optimizers import Adam
# from sklearn.preprocessing import LabelEncoder
# from transformers import BertTokenizer, TFBertForSequenceClassification
# from transformers import InputExample, InputFeatures
# import tensorflow as tf
# import pandas as pd
# import os
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from transformers import BertTokenizer, TFBertForSequenceClassification
# from transformers import InputExample, InputFeatures
# import tensorflow as tf
# from tensorflow.keras.optimizers import Adam
import tensorflow as tf
# from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd


In [ ]:
import numpy as np
import random
import torch

seed_val = 42

np.random.seed(seed_val)
torch.manual_seed(seed_val)
random.seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

## **Binary Classification**

In [ ]:
import pandas as pd
import os

folder_path = '/content/'
all_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]
df_list = [pd.read_csv(file) for file in all_files]
df = pd.concat(df_list, ignore_index=True)

In [ ]:
df.head()

,sentences,labels
0,this appeal by special leave arises out of a j...,0
1,rendered by the high court of judicature at bo...,0
2,in order to appreciate the issue requiring det...,0
3,the appellant m s bharat petroleum corporation...,1
4,the respondent m s great eastern shipping comp...,1


In [ ]:
df.shape

(6519, 2)

In [ ]:
def apply_rf_classifier_grid_search(df, test_split=0.2, multi_class = False):
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score, classification_report
    from sklearn.model_selection import GridSearchCV
    from sklearn.preprocessing import LabelEncoder

    X = df['sentences']
    y = df['labels']

    if multi_class:
        label_encoder = LabelEncoder()
        y = label_encoder.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    vectorizer = TfidfVectorizer()
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    rf = RandomForestClassifier()
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 30, 50, 100],
    }
    grid_search = GridSearchCV(rf, param_grid, cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    rf = grid_search.best_estimator_
    y_pred = rf.predict(X_test)

    print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
    print(classification_report(y_test, y_pred))
    print("Best parameters: ", grid_search.best_params_)

    if multi_class:
        print("Labels: ", label_encoder.classes_)
        return rf, label_encoder, grid_search.best_params_
    else:
        return rf, vectorizer, grid_search.best_params_

def apply_svm_grid_search(df, test_split=0.2, multi_class = False):
    from sklearn.svm import SVC
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import accuracy_score, classification_report
    from sklearn.model_selection import GridSearchCV
    from sklearn.preprocessing import LabelEncoder

    X = df['sentences']
    y = df['labels']

    if multi_class:
        label_encoder = LabelEncoder()
        y = label_encoder.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_split, random_state=42)

    vectorizer = TfidfVectorizer()
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    svm = SVC()
    param_grid = {
        'C': [0.1, 1, 10],
        'gamma': [1, 0.1, 0.01],
        'kernel': ['rbf', 'poly', 'sigmoid']
    }
    grid_search = GridSearchCV(svm, param_grid, cv=5, n_jobs=-1)
    grid_search.fit(X_train, y_train)
    svm = grid_search.best_estimator_
    y_pred = svm.predict(X_test)

    print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
    print(classification_report(y_test, y_pred))
    print("Best parameters: ", grid_search.best_params_)

    if multi_class:
        print("Labels: ", label_encoder.classes_)
        return svm, label_encoder, grid_search.best_params_
    else:
        return svm, vectorizer, grid_search.best_params_

def apply_lstm_classifier(df, test_split=0.2, multi_class = False):
    from tensorflow.keras.preprocessing.text import Tokenizer
    from keras.preprocessing.sequence import pad_sequences
    from keras.models import Sequential
    from keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout, Flatten
    from keras.optimizers import Adam
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import accuracy_score, classification_report
    import numpy as np

    X = df['sentences']
    y = df['labels']

    # if multi_class:
    #     label_encoder = LabelEncoder()
    #     y = label_encoder.fit_transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    label_encoder = LabelEncoder()
    y_train = label_encoder.fit_transform(y_train)
    y_test = label_encoder.transform(y_test)

    tokenizer = Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(X_train)

    X_train = tokenizer.texts_to_sequences(X_train)
    X_test = tokenizer.texts_to_sequences(X_test)

    vocab_size = len(tokenizer.word_index) + 1
    maxlen = 100

    X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
    X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=maxlen))
    model.add(Conv1D(128, 5, activation='relu'))
    model.add(MaxPooling1D(5))
    model.add(Bidirectional(LSTM(100)))
    model.add(Flatten())
    if multi_class:
        model.add(Dense(len(label_encoder.classes_), activation='softmax'))
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    else:
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

    print(model.summary())
    print(model.evaluate(X_test, y_test))

    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    print(classification_report(y_test, y_pred))


    return model, X_test, y_test, label_encoder

def apply_fcnn_classifier(df, test_split=0.2, multi_class=False):
    from tensorflow.keras.preprocessing.text import Tokenizer
    from keras.preprocessing.sequence import pad_sequences
    from keras.models import Sequential
    from keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Bidirectional, Dense, Dropout, Flatten
    from keras.optimizers import Adam
    from sklearn.model_selection import train_test_split
    from sklearn.preprocessing import LabelEncoder
    from sklearn.metrics import accuracy_score, classification_report
    import numpy as np

    X = df['sentences']
    y = df['labels']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_split, random_state=42)

    label_encoder = LabelEncoder()
    y_train = label_encoder.fit_transform(y_train)
    y_test = label_encoder.transform(y_test)

    tokenizer = Tokenizer(num_words=5000)
    tokenizer.fit_on_texts(X_train)

    X_train = tokenizer.texts_to_sequences(X_train)
    X_test = tokenizer.texts_to_sequences(X_test)

    vocab_size = len(tokenizer.word_index) + 1
    maxlen = 100

    X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
    X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=maxlen))
    model.add(Conv1D(128, 5, activation='relu'))
    model.add(MaxPooling1D(5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Flatten())
    if multi_class:
        model.add(Dense(len(label_encoder.classes_), activation='softmax'))
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    else:
        model.add(Dense(1, activation='sigmoid'))
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    print(model.summary())

    model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

    print(model.evaluate(X_test, y_test))

    y_pred = model.predict(X_test)
    y_pred = np.argmax(y_pred, axis=1)
    print(classification_report(y_test, y_pred))

    return model, tokenizer, label_encoder

In [ ]:
model, vectorizer_lr, params = apply_rf_classifier_grid_search(df)

Accuracy: 0.7354294478527608
              precision    recall  f1-score   support

           0       0.74      1.00      0.85       957
           1       1.00      0.01      0.01       347

    accuracy                           0.74      1304
   macro avg       0.87      0.50      0.43      1304
weighted avg       0.81      0.74      0.62      1304

Best parameters:  {'max_depth': 50, 'n_estimators': 100}


In [ ]:
model, vectorizer_lr, params = apply_svm_grid_search(df)

Accuracy: 0.7361963190184049
              precision    recall  f1-score   support

           0       0.74      0.99      0.85       957
           1       0.56      0.04      0.08       347

    accuracy                           0.74      1304
   macro avg       0.65      0.52      0.46      1304
weighted avg       0.69      0.74      0.64      1304

Best parameters:  {'C': 10, 'gamma': 1, 'kernel': 'poly'}


In [ ]:
model, X_test, y_test, label_encoder = apply_lstm_classifier(df)

Epoch 1/10
163/163 [==============================] - 37s 201ms/step - loss: 0.5616 - accuracy: 0.7417 - val_loss: 0.5431 - val_accuracy: 0.7546
Epoch 2/10
163/163 [==============================] - 32s 194ms/step - loss: 0.4600 - accuracy: 0.7862 - val_loss: 0.5583 - val_accuracy: 0.7446
Epoch 3/10
163/163 [==============================] - 33s 200ms/step - loss: 0.3471 - accuracy: 0.8485 - val_loss: 0.6574 - val_accuracy: 0.6979
Epoch 4/10
163/163 [==============================] - 31s 193ms/step - loss: 0.2506 - accuracy: 0.8989 - val_loss: 0.7538 - val_accuracy: 0.7163
Epoch 5/10
163/163 [==============================] - 31s 193ms/step - loss: 0.1622 - accuracy: 0.9402 - val_loss: 1.0178 - val_accuracy: 0.6879
Epoch 6/10
163/163 [==============================] - 34s 210ms/step - loss: 0.1128 - accuracy: 0.9597 - val_loss: 1.1224 - val_accuracy: 0.6833
Epoch 7/10
163/163 [==============================] - 31s 193ms/step - loss: 0.0700 - accuracy: 0.9762 - val_loss: 1.3082 - val_ac

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model, tokenizer, label_encoder = apply_fcnn_classifier(df)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 100, 100)          974400    
                                                                 
 conv1d_1 (Conv1D)           (None, 96, 128)           64128     
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 19, 128)           0         
 g1D)                                                            
                                                                 
 dense_1 (Dense)             (None, 19, 128)           16512     
                                                                 
 dropout (Dropout)           (None, 19, 128)           0         
                                                                 
 dense_2 (Dense)             (None, 19, 64)            8256      
                                                      

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

features = df['sentences']
labels = df['labels']

train_features, val_features, train_labels, val_labels = train_test_split(
    features, labels, test_size=0.2, random_state=42)

batch_size = 8

train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((val_features, val_labels)).batch(batch_size)

In [ ]:
print(next(iter(train_dataset)))

(<tf.Tensor: shape=(8,), dtype=string, numpy=
array([b'in inviting your attention to the reference cited i am to inform you that government regret their inability to comply with your request.',
       b'as already stated the additional sessions judge convicted both the respondents the respondent No i in respect of each head of the offences with which he was charged and the respondent No 2 in respect only of the offences of conspiracy and misappropriation.',
       b'unfortunately for the prosecution there is no evidence to show that in fact the wearing apparels containing bloodstains belonged to the accused save and except the alleged confessional statement.',
       b's 37 deals with persons not entitled under s 34 to be deemed in certain circumstances as protected tenants.',
       b'ss 116 and 117a were reworded which read as under s 116 1 subject to the provisions of this act the appellate board established u s 83 of the trade marks act1999 shall be the appellate board for the purp

In [ ]:
import keras_nlp

classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_base_en_uncased",
    num_classes=2,
)

classifier.fit(train_dataset, validation_data=val_dataset, epochs=3, verbose = 1)

Epoch 1/3
652/652 ━━━━━━━━━━━━━━━━━━━━ 765s 937ms/step - loss: 0.5719 - sparse_categorical_accuracy: 0.7472 - val_loss: 0.5859 - val_sparse_categorical_accuracy: 0.7546
Epoch 2/3
652/652 ━━━━━━━━━━━━━━━━━━━━ 504s 772ms/step - loss: 0.5735 - sparse_categorical_accuracy: 0.7482 - val_loss: 0.6274 - val_sparse_categorical_accuracy: 0.7546
Epoch 3/3
652/652 ━━━━━━━━━━━━━━━━━━━━ 510s 782ms/step - loss: 0.5743 - sparse_categorical_accuracy: 0.7464 - val_loss: 0.6132 - val_sparse_categorical_accuracy: 0.7546


In [ ]:
# save the classifier
classifier.save('bert_3_classifier.keras')

In [ ]:
# take the entire val_dataset
# text, labels = next(iter(val_dataset))
y_pred = classifier.predict(val_dataset)
y_pred

163/163 ━━━━━━━━━━━━━━━━━━━━ 36s 217ms/step


array([[-0.21688384, -0.604051  ],
       [-0.2168372 , -0.60402215],
       [-0.21686131, -0.60403687],
       ...,
       [-0.21685566, -0.60403323],
       [-0.21685576, -0.6040323 ],
       [-0.21687275, -0.6040438 ]], dtype=float32)

In [ ]:
y_pred_labels = np.argmax(y_pred, axis=1)
y_pred_labels

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
len(y_pred_labels)

1304

In [ ]:
len(val_labels)

1304

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(val_labels, y_pred_labels))

              precision    recall  f1-score   support

           0       0.75      1.00      0.86       984
           1       0.00      0.00      0.00       320

    accuracy                           0.75      1304
   macro avg       0.38      0.50      0.43      1304
weighted avg       0.57      0.75      0.65      1304



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## **Multiclass Classification**

In [ ]:
import pandas as pd
import os

folder_path = '/content/drive/MyDrive/NLP SEM 5/CSVs/train_data'
all_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]
print(f"No. of files: {len(all_files)}")
df_list = [pd.read_csv(file) for file in all_files]
df = pd.concat(df_list, ignore_index=True)
df.columns = ['sentences', 'labels']

No. of files: 50


In [ ]:
df.head()

,sentences,labels
0,one lakshminarayana iyer a hindu brahmin who o...,none
1,ramalakshmi had married the plaintiff and had ...,facts
2,they were all alive in december 1924 when laks...,facts
3,before his death he executed a will on 16th no...,none
4,by this will he gave the following directions ...,none


In [ ]:
df.shape

(8656, 2)

In [ ]:
model, vectorizer_lr, params = apply_rf_classifier_grid_search(df, multi_class=True)

Accuracy: 0.7829099307159353
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       204
           1       0.00      0.00      0.00        32
           2       0.00      0.00      0.00       112
           3       0.50      0.21      0.30        28
           4       0.78      1.00      0.88      1356

    accuracy                           0.78      1732
   macro avg       0.26      0.24      0.24      1732
weighted avg       0.62      0.78      0.69      1732

Best parameters:  {'max_depth': 100, 'n_estimators': 200}
Labels:  ['analysis' 'argument' 'facts' 'judgement' 'none']


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
model, lable_encoder, params = apply_svm_grid_search(df, multi_class=True)
print(label_encoder.classes_)

/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


Accuracy: 0.7811778290993071
              precision    recall  f1-score   support

           0       0.35      0.05      0.09       204
           1       0.33      0.03      0.06        32
           2       0.78      0.06      0.12       112
           3       0.48      0.43      0.45        28
           4       0.79      0.97      0.88      1356

    accuracy                           0.78      1732
   macro avg       0.55      0.31      0.32      1732
weighted avg       0.73      0.78      0.71      1732

Best parameters:  {'C': 10, 'gamma': 1, 'kernel': 'rbf'}
Labels:  ['analysis' 'argument' 'facts' 'judgement' 'none']
['analysis' 'argument' 'facts' 'judgement' 'none']


In [ ]:
model, x_test, y_test, label_encoder = apply_lstm_classifier(df, multi_class=True)
print(label_encoder.classes_)

Epoch 1/10
217/217 [==============================] - 44s 162ms/step - loss: 0.7787 - accuracy: 0.7724 - val_loss: 0.7218 - val_accuracy: 0.7829
Epoch 2/10
217/217 [==============================] - 35s 160ms/step - loss: 0.6200 - accuracy: 0.7844 - val_loss: 0.6977 - val_accuracy: 0.7812
Epoch 3/10
217/217 [==============================] - 24s 109ms/step - loss: 0.4226 - accuracy: 0.8445 - val_loss: 0.8258 - val_accuracy: 0.7569
Epoch 4/10
217/217 [==============================] - 21s 97ms/step - loss: 0.2632 - accuracy: 0.9077 - val_loss: 1.0468 - val_accuracy: 0.6934
Epoch 5/10
217/217 [==============================] - 22s 101ms/step - loss: 0.1310 - accuracy: 0.9551 - val_loss: 1.2786 - val_accuracy: 0.7477
Epoch 6/10
217/217 [==============================] - 24s 113ms/step - loss: 0.0539 - accuracy: 0.9831 - val_loss: 1.5645 - val_accuracy: 0.7206
Epoch 7/10
217/217 [==============================] - 21s 99ms/step - loss: 0.0238 - accuracy: 0.9928 - val_loss: 1.7898 - val_accu

In [ ]:
model, tokenizer, label_encoder = apply_fcnn_classifier(df, multi_class=True)
print(label_encoder.classes_)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 100, 100)          1115500   
                                                                 
 conv1d_3 (Conv1D)           (None, 96, 128)           64128     
                                                                 
 max_pooling1d_3 (MaxPoolin  (None, 19, 128)           0         
 g1D)                                                            
                                                                 
 dense_9 (Dense)             (None, 19, 128)           16512     
                                                                 
 dropout_6 (Dropout)         (None, 19, 128)           0         
                                                                 
 dense_10 (Dense)            (None, 19, 64)            8256      
                                                      

In [ ]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
features = df['sentences']
labels = df['labels']

encoder = LabelEncoder()
labels = encoder.fit_transform(labels)

train_features, val_features, train_labels, val_labels = train_test_split(
    features, labels, test_size=0.2, random_state=42)

batch_size = 8

train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((val_features, val_labels)).batch(batch_size)

In [ ]:
import keras_nlp

multi_classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_base_en_uncased",
    num_classes=len(encoder.classes_),
)

multi_classifier.fit(train_dataset, validation_data=val_dataset, epochs=3, verbose=1)

100%|██████████| 510/510 [00:00<00:00, 1.26MB/s]
100%|██████████| 418M/418M [00:08<00:00, 51.4MB/s]
100%|██████████| 547/547 [00:00<00:00, 646kB/s]
100%|██████████| 226k/226k [00:00<00:00, 1.19MB/s]


Epoch 1/3
866/866 ━━━━━━━━━━━━━━━━━━━━ 1054s 1s/step - loss: 0.7881 - sparse_categorical_accuracy: 0.7805 - val_loss: 0.6628 - val_sparse_categorical_accuracy: 0.7829
Epoch 2/3
866/866 ━━━━━━━━━━━━━━━━━━━━ 825s 953ms/step - loss: 0.6550 - sparse_categorical_accuracy: 0.7874 - val_loss: 0.6276 - val_sparse_categorical_accuracy: 0.7835
Epoch 3/3
866/866 ━━━━━━━━━━━━━━━━━━━━ 761s 879ms/step - loss: 0.5471 - sparse_categorical_accuracy: 0.8040 - val_loss: 0.6428 - val_sparse_categorical_accuracy: 0.7875


In [ ]:
from sklearn.metrics import classification_report
import numpy as np
y_pred = multi_classifier.predict(val_dataset)
y_pred_labels = np.argmax(y_pred, axis=1)
print(classification_report(val_labels, y_pred_labels))
print(encoder.classes_)

217/217 ━━━━━━━━━━━━━━━━━━━━ 59s 270ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       204
           1       0.00      0.00      0.00        32
           2       0.53      0.19      0.28       112
           3       0.69      0.32      0.44        28
           4       0.79      0.98      0.88      1356

    accuracy                           0.79      1732
   macro avg       0.40      0.30      0.32      1732
weighted avg       0.67      0.79      0.71      1732

['analysis' 'argument' 'facts' 'judgement' 'none']


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

# Assuming your data is in a pandas DataFrame with 'sentences' and 'labels' columns
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Prepare the dataset
texts = df['sentences'].values
labels = df['labels'].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Create datasets
train_dataset = TextDataset(X_train, y_train, tokenizer, max_len=128)
test_dataset = TextDataset(X_test, y_test, tokenizer, max_len=128)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy="epoch"
)

# Define a compute metrics function
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    return classification_report(labels, pred, output_dict=True)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,0,1,Accuracy,Macro avg,Weighted avg
1,0.510100,0.529593,"{'precision': 0.7648780487804878, 'recall': 1.0, 'f1-score': 0.8667772249861803, 'support': 784}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 241}",0.764878,"{'precision': 0.3824390243902439, 'recall': 0.5, 'f1-score': 0.43338861249309013, 'support': 1025}","{'precision': 0.5850384295062463, 'recall': 0.7648780487804878, 'f1-score': 0.6629788725747955, 'support': 1025}"
2,0.551300,0.526729,"{'precision': 0.7777777777777778, 'recall': 0.9821428571428571, 'f1-score': 0.8680947012401352, 'support': 784}","{'precision': 0.6, 'recall': 0.08713692946058091, 'f1-score': 0.15217391304347824, 'support': 241}",0.771707,"{'precision': 0.6888888888888889, 'recall': 0.534639893301719, 'f1-score': 0.5101343071418067, 'support': 1025}","{'precision': 0.7359783197831979, 'recall': 0.7717073170731708, 'f1-score': 0.6997660086007261, 'support': 1025}"
3,0.314900,0.608243,"{'precision': 0.8293650793650794, 'recall': 0.7997448979591837, 'f1-score': 0.8142857142857144, 'support': 784}","{'precision': 0.4163568773234201, 'recall': 0.46473029045643155, 'f1-score': 0.4392156862745098, 'support': 241}",0.720976,"{'precision': 0.6228609783442498, 'recall': 0.6322375942078076, 'f1-score': 0.626750700280112, 'support': 1025}","{'precision': 0.732257785031382, 'recall': 0.7209756097560975, 'f1-score': 0.7260985174557629, 'support': 1025}"


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.7648780

TrainOutput(global_step=771, training_loss=0.49595266376198033, metrics={'train_runtime': 308.678, 'train_samples_per_second': 39.838, 'train_steps_per_second': 2.498, 'total_flos': 808869161940480.0, 'train_loss': 0.49595266376198033, 'epoch': 3.0})

In [ ]:
# Evaluate the model
results = trainer.evaluate()
print(f"Accuracy: {results['eval_accuracy']}")

# Get predictions
predictions = trainer.predict(test_dataset)
y_pred = np.argmax(predictions.predictions, axis=1)

# Generate classification report
print(classification_report(y_test, y_pred))

Trainer is attempting to log a value of "{'precision': 0.8293650793650794, 'recall': 0.7997448979591837, 'f1-score': 0.8142857142857144, 'support': 784}" of type <class 'dict'> for key "eval/0" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.4163568773234201, 'recall': 0.46473029045643155, 'f1-score': 0.4392156862745098, 'support': 241}" of type <class 'dict'> for key "eval/1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.6228609783442498, 'recall': 0.6322375942078076, 'f1-score': 0.626750700280112, 'support': 1025}" of type <class 'dict'> for key "eval/macro avg" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.732257785031382, 'recall'

Accuracy: 0.7209756097560975
              precision    recall  f1-score   support

           0       0.83      0.80      0.81       784
           1       0.42      0.46      0.44       241

    accuracy                           0.72      1025
   macro avg       0.62      0.63      0.63      1025
weighted avg       0.73      0.72      0.73      1025



# bert with class imbalance



In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.utils.class_weight import compute_class_weight


class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Initialize tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Prepare the dataset
texts = df['sentences'].values
labels = df['labels'].values

# Split the data
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Create datasets
train_dataset = TextDataset(X_train, y_train, tokenizer, max_len=128)
test_dataset = TextDataset(X_test, y_test, tokenizer, max_len=128)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# Compute class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float)

# Update the model to use class weights in the loss function
class BertForSequenceClassificationWithWeightedLoss(BertForSequenceClassification):
    def forward(self, input_ids, attention_mask=None, token_type_ids=None, position_ids=None, head_mask=None, labels=None):
        outputs = super().forward(input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, position_ids=position_ids, head_mask=head_mask, labels=labels)
        if labels is not None:
            loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights.to(input_ids.device))
            loss = loss_fct(outputs.logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs[2:]
        return outputs

model = BertForSequenceClassificationWithWeightedLoss.from_pretrained('bert-base-uncased', num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Define a compute metrics function
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    return classification_report(labels, pred, output_dict=True)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

# Evaluate the model
trainer.evaluate()

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of BertForSequenceClassificationWithWeightedLoss were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:

# Get predictions and generate classification report
predictions = trainer.predict(test_dataset)
pred_labels = np.argmax(predictions.predictions, axis=1)
print(classification_report(y_test, pred_labels))

# roberta-large

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
x
# Custom dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }


texts = df['sentences'].values
labels = df['labels'].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Initialize tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
model = RobertaForSequenceClassification.from_pretrained('roberta-large', num_labels=2)

# Create datasets
train_dataset = TextDataset(X_train, y_train, tokenizer, max_len=128)
test_dataset = TextDataset(X_test, y_test, tokenizer, max_len=128)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Define a compute metrics function
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    return classification_report(labels, pred, output_dict=True)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,0,1,Accuracy,Macro avg,Weighted avg
1,0.641400,0.545493,"{'precision': 0.7648780487804878, 'recall': 1.0, 'f1-score': 0.8667772249861803, 'support': 784}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 241}",0.764878,"{'precision': 0.3824390243902439, 'recall': 0.5, 'f1-score': 0.43338861249309013, 'support': 1025}","{'precision': 0.5850384295062463, 'recall': 0.7648780487804878, 'f1-score': 0.6629788725747955, 'support': 1025}"
2,0.535200,0.549830,"{'precision': 0.7648780487804878, 'recall': 1.0, 'f1-score': 0.8667772249861803, 'support': 784}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 241}",0.764878,"{'precision': 0.3824390243902439, 'recall': 0.5, 'f1-score': 0.43338861249309013, 'support': 1025}","{'precision': 0.5850384295062463, 'recall': 0.7648780487804878, 'f1-score': 0.6629788725747955, 'support': 1025}"
3,0.541400,0.548830,"{'precision': 0.7648780487804878, 'recall': 1.0, 'f1-score': 0.8667772249861803, 'support': 784}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 241}",0.764878,"{'precision': 0.3824390243902439, 'recall': 0.5, 'f1-score': 0.43338861249309013, 'support': 1025}","{'precision': 0.5850384295062463, 'recall': 0.7648780487804878, 'f1-score': 0.6629788725747955, 'support': 1025}"


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.7648780

TrainOutput(global_step=1539, training_loss=0.5947475805152462, metrics={'train_runtime': 1205.1142, 'train_samples_per_second': 10.204, 'train_steps_per_second': 1.277, 'total_flos': 2864989993711104.0, 'train_loss': 0.5947475805152462, 'epoch': 3.0})

In [ ]:
# Evaluate the model
results = trainer.evaluate()
print(f"Accuracy: {results['eval_accuracy']}")

# Get predictions
predictions = trainer.predict(test_dataset)
y_pred = np.argmax(predictions.predictions, axis=1)

# Generate classification report
print(classification_report(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.7648780

Accuracy: 0.7648780487804878


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.76      1.00      0.87       784
           1       0.00      0.00      0.00       241

    accuracy                           0.76      1025
   macro avg       0.38      0.50      0.43      1025
weighted avg       0.59      0.76      0.66      1025



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# deberta


In [ ]:
from transformers import DebertaTokenizer, DebertaForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Custom dataset class
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

# Load data
# df = pd.read_csv('your_dataset.csv')  # replace with your dataset
texts = df['sentences'].values
labels = df['labels'].values

# Split data
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Initialize tokenizer and model
tokenizer = DebertaTokenizer.from_pretrained('microsoft/deberta-base')
model = DebertaForSequenceClassification.from_pretrained('microsoft/deberta-base', num_labels=2)

# Create datasets
train_dataset = TextDataset(X_train, y_train, tokenizer, max_len=128)
test_dataset = TextDataset(X_test, y_test, tokenizer, max_len=128)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,  # You can reduce this to 1 or 2 for faster experimentation
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Define a compute metrics function
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    return classification_report(labels, pred, output_dict=True)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

# Train the model
trainer.train()


Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss


In [ ]:

# Evaluate the model
results = trainer.evaluate()
print(f"Accuracy: {results['eval_accuracy']}")

# Get predictions
predictions = trainer.predict(test_dataset)
y_pred = np.argmax(predictions.predictions, axis=1)

# Generate classification report
print(classification_report(y_test, y_pred))


Epoch,Training Loss,Validation Loss,0,1,Accuracy,Macro avg,Weighted avg
0,No log,0.656894,"{'precision': 0.7648780487804878, 'recall': 1.0, 'f1-score': 0.8667772249861803, 'support': 784}","{'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 241}",0.764878,"{'precision': 0.3824390243902439, 'recall': 0.5, 'f1-score': 0.43338861249309013, 'support': 1025}","{'precision': 0.5850384295062463, 'recall': 0.7648780487804878, 'f1-score': 0.6629788725747955, 'support': 1025}"


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'precision': 0.7648780

Accuracy: 0.7648780487804878
              precision    recall  f1-score   support

           0       0.76      1.00      0.87       784
           1       0.00      0.00      0.00       241

    accuracy                           0.76      1025
   macro avg       0.38      0.50      0.43      1025
weighted avg       0.59      0.76      0.66      1025



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classif

In [ ]:
# Preprocess the text data
max_words = 10000  # Maximum number of words to tokenize
maxlen = 100  # Maximum length of sequences

In [ ]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['sentences'])
sequences = tokenizer.texts_to_sequences(df['sentences'])
X = pad_sequences(sequences, maxlen=maxlen)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df['labels'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
model = Sequential()
model.add(Embedding(max_words, 128, input_length=maxlen))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
103/103 [==============================] - 20s 138ms/step - loss: 0.5794 - accuracy: 0.7438 - val_loss: 0.5819 - val_accuracy: 0.7244
Epoch 2/10
103/103 [==============================] - 17s 161ms/step - loss: 0.5233 - accuracy: 0.7527 - val_loss: 0.5903 - val_accuracy: 0.7232
Epoch 3/10
103/103 [==============================] - 13s 124ms/step - loss: 0.3235 - accuracy: 0.8676 - val_loss: 0.7007 - val_accuracy: 0.6878
Epoch 4/10
103/103 [==============================] - 12s 112ms/step - loss: 0.1342 - accuracy: 0.9515 - val_loss: 0.9188 - val_accuracy: 0.6915
Epoch 5/10
103/103 [==============================] - 13s 128ms/step - loss: 0.0594 - accuracy: 0.9817 - val_loss: 1.1255 - val_accuracy: 0.6854
Epoch 6/10
103/103 [==============================] - 15s 143ms/step - loss: 0.0251 - accuracy: 0.9939 - val_loss: 1.3993 - val_accuracy: 0.6951
Epoch 7/10
103/103 [==============================] - 13s 128ms/step - loss: 0.0152 - accuracy: 0.9957 - val_loss: 1.5774 - val_ac

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')

33/33 [==============================] - 1s 21ms/step - loss: 1.7262 - accuracy: 0.6966
Accuracy: 0.6965853571891785


In [ ]:
from sklearn.metrics import classification_report

y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))


33/33 [==============================] - 2s 18ms/step
              precision    recall  f1-score   support

           0       0.80      0.81      0.80       784
           1       0.34      0.32      0.33       241

    accuracy                           0.70      1025
   macro avg       0.57      0.57      0.57      1025
weighted avg       0.69      0.70      0.69      1025



In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['sentences'], df['labels'], test_size=0.2, random_state=42)


In [ ]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [ ]:
#class imbalance
# Logistic Regression with class weight balancing
logistic_model = LogisticRegression(class_weight='balanced', random_state=42)
logistic_model.fit(X_train_tfidf, y_train)
y_pred_logistic = logistic_model.predict(X_test_tfidf)

# Evaluate Logistic Regression model
accuracy_logistic = accuracy_score(y_test, y_pred_logistic)
print(f'Logistic Regression Accuracy: {accuracy_logistic}')
print('Logistic Regression Classification Report:')
print(classification_report(y_test, y_pred_logistic))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

LogisticRegression()

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_tfidf, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
y_pred = model.predict(X_test_tfidf)


In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print('Classification Report:')
print(classification_report(y_test, y_pred))

Accuracy: 0.775609756097561
Classification Report:
              precision    recall  f1-score   support

           0       0.77      1.00      0.87       784
           1       0.87      0.05      0.10       241

    accuracy                           0.78      1025
   macro avg       0.82      0.53      0.49      1025
weighted avg       0.80      0.78      0.69      1025

